In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/My Drive/cs109_cassie'

/content/drive/My Drive/cs109_cassie


# Get Similiar Users Ratings and Map Back to Dataframe

In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
import re
import numpy as np
import ast

In [2]:
cleaned_df = pd.read_csv('data/cleaned_df.csv')
user_lda_df = pd.read_csv('data/user_reviews_lda_df.csv')

In [3]:
user_lda_df['topic_vector'] = user_lda_df['topic_vector'].apply(ast.literal_eval)
user_lda_df.head()


,user_id,text,processed_text,topic_vector
0,1.000018e+20,Great food. The service was a little slow. It ...,great food service little slow nice clean food...,"[0.009604099033542457, 0.009604097040164497, 0..."
1,1.000020e+20,We had a large party but they accommodated us ...,large party accommodated could watch patriot g...,"[0.9332929748068605, 0.004764784389977953, 0.0..."
2,1.000027e+20,Margarita was yummy. Food not great.\nWe love ...,margarita yummy food great love tasty burger w...,"[0.0068352206810172785, 0.00683521500500783, 0..."
3,1.000030e+20,I eat there when I have a chance to do it I ha...,eat chance going since kid go area love food w...,"[0.011245047569970199, 0.011245043923995425, 0..."
4,1.000057e+20,Drinks desert and Jarvis tops off a good eveni...,drink desert jarvis top good evening tewkesbur...,"[0.005380025279412345, 0.005380015639471415, 0..."


In [4]:
cleaned_df.head()


,user_id,name_y,rating,text,gmap_id,latitude,longitude,num_of_reviews,price,avg_rating
0,1.182288e+20,Popeyes Louisiana Kitchen,3,Love the chicken sandwiches but the service ne...,0x89e4f921a38f0b11:0x879ccb1d12f010de,41.695425,-71.140807,526,$,3.6
1,1.090050e+20,Popeyes Louisiana Kitchen,3,The place was super smokey when we came in whi...,0x89e4f921a38f0b11:0x879ccb1d12f010de,41.695425,-71.140807,526,$,3.6
2,1.128417e+20,Popeyes Louisiana Kitchen,5,"While I love all their chicken offerings, my w...",0x89e4f921a38f0b11:0x879ccb1d12f010de,41.695425,-71.140807,526,$,3.6
3,1.175200e+20,Popeyes Louisiana Kitchen,5,Always the best fried chicken around. Mash pot...,0x89e4f921a38f0b11:0x879ccb1d12f010de,41.695425,-71.140807,526,$,3.6
4,1.130399e+20,Popeyes Louisiana Kitchen,5,Awesome chicken crisp not greasy!,0x89e4f921a38f0b11:0x879ccb1d12f010de,41.695425,-71.140807,526,$,3.6


In [5]:
user_restaurants_df = cleaned_df.groupby('user_id')['gmap_id'].agg(list).reset_index()

# Rename the column for clarity
user_restaurants_df.rename(columns={'gmap_id': 'reviewed_restaurant_id'}, inplace=True)

# Display the first few rows of the new DataFrame to verify
user_restaurants_df

,user_id,reviewed_restaurant_id
0,1.000018e+20,"[0x89e3f01c0c712697:0xd5104ac7a3f9aaa0, 0x89e3..."
1,1.000020e+20,"[0x89e4845fa138d201:0xcfa13f1f42e5c64b, 0x89e3..."
2,1.000027e+20,"[0x89e3a3cbc21e11bf:0x273ba9950150ec5d, 0x89e3..."
3,1.000030e+20,"[0x89e4067ca5807965:0xf7dc88f1bc8f32e6, 0x89e4..."
4,1.000057e+20,"[0x89e3a0bbc3ba2883:0xd6dc5a55202a9754, 0x89e3..."
...,...,...
10938,1.184382e+20,"[0x89e30ae7eccf2433:0x2cf9872bf7e5855c, 0x89e3..."
10939,1.184401e+20,"[0x89e370b8f913c249:0x1a3621091fb89d1e, 0x89e3..."
10940,1.184433e+20,"[0x89e37a0fbbb97dd9:0x65ac692307206f18, 0x89e3..."
10941,1.184449e+20,"[0x89e3712742d15c25:0x10eb19a5ab960482, 0x89e3..."


In [6]:
user_lda_df['topic_vector'][0]

[0.009604099033542457,
 0.009604097040164497,
 0.009604101060079497,
 0.009604079099371443,
 0.00960407311987992,
 0.009604097631770385,
 0.8655427607911149,
 0.00960407451847766,
 0.009604087103588337,
 0.009604100357096638,
 0.00960407394057977,
 0.009604073185919012,
 0.009604099964343048,
 0.009604073227705023,
 0.009604109926367448]

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

embeddings = np.array(user_lda_df['topic_vector'].tolist())
similarity_matrix = cosine_similarity(embeddings)

In [15]:
def get_similar_user_ratings(similarity_matrix, user_df, user_restaurants_df, embeddings, cleaned_df, target_user_id, target_restaurant_id, top_n=10):
    similarity_matrix = similarity_matrix
    user_lda_df = user_df
    user_restaurants_df = user_restaurants_df
    embeddings = embeddings
    cleaned_df = cleaned_df



    # Get index of the target user
    target_index = user_lda_df[user_lda_df['user_id'] == target_user_id].index[0]

    # Get similarity scores for this user against all others
    similarity_scores = similarity_matrix[target_index]

    # Create a DataFrame from similarity scores
    similar_users = pd.DataFrame({
        'user_id': user_lda_df['user_id'],
        'similarity_score': similarity_scores
    })

    # Filter out the target user
    similar_users = similar_users[similar_users['user_id'] != target_user_id]

    # Sort users by similarity score in descending order
    similar_users = similar_users.sort_values(by='similarity_score', ascending=False)

    # Filter users who have reviewed the target restaurant
    reviewed_same_restaurant = similar_users[similar_users['user_id'].isin(user_restaurants_df[user_restaurants_df['reviewed_restaurant_id'].apply(lambda x: target_restaurant_id in x)]['user_id'])]

    # Get the top N similar users who have reviewed the restaurant
    top_similar_users = reviewed_same_restaurant.head(top_n)

    # Fetch ratings for these users for the target restaurant
    ratings = []
    for _, row in top_similar_users.iterrows():
        user_rating = cleaned_df[(cleaned_df['user_id'] == row['user_id']) & (cleaned_df['gmap_id'] == target_restaurant_id)]['rating'].iloc[0]
        ratings.append((user_rating, row['similarity_score']))

    # Calculate weighted average rating
    if ratings:
        weighted_rating = sum(r * w for r, w in ratings) / sum(w for _, w in ratings)
        # Clamp the rating to be within 1 to 5
        return max(min(weighted_rating, 5), 1)
    else:
        return None


In [16]:


# Apply the function for each row in cleaned_df
cleaned_df['similar_user_rating'] = cleaned_df.apply(lambda row: get_similar_user_ratings(similarity_matrix=similarity_matrix,user_df = user_lda_df, user_restaurants_df=user_restaurants_df, embeddings=embeddings, cleaned_df=cleaned_df, target_user_id= row['user_id'], target_restaurant_id= row['gmap_id']), axis=1)


In [ ]:
cleaned_df

In [ ]:
cleaned_df.to_csv("data/filled_lda_df.csv", index=False)

## Predictive Modeling: XGBoost

In [ ]:
df_cleaned = pd.read_csv('data/filled_lda_df')
one_hot = pd.get_dummies(df_cleaned['price'])
df_cleaned = pd.concat([df_cleaned, one_hot], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_cleaned.drop(['rating','text','name_y','price','user_id','gmap_id'], axis=1), 
                                                    df_cleaned[['rating']], 
                                                    test_size=0.2, random_state = 42)
X_train

In [ ]:
import xgboost as xgb
model1 = xgb.XGBRegressor(objective='reg:squarederror')
model1.fit(X_train, y_train)

In [ ]:
pred1 = model1.predict(X_test)
rmse = np.sqrt(np.mean((pred1 - y_test.to_numpy())**2))
print(f'content-based rmse = {rmse}')

In [ ]:
feature_importance = pd.DataFrame(model1.feature_importances_, index=X_train.columns, columns=['Importance'])
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

print(feature_importance)